In [ ]:
%matplotlib widget

import re

from bokeh.models import Band, ColumnDataSource, HoverTool, Range1d
from bokeh.plotting import figure, output_file, output_notebook, show
import pandas as pd
import numpy as np
import requests

from nena_api import (
    get_series_data,
    obtain_access_token,
    retrieve_metadata_by_prefix,
)

## Hydro reservoir visualization tutorial.

In this tutorial we will showcase how to plot the hydro reservoir filling in Norway using the historical
data available from the Nena API. The tutorial uses a set of function wrappers around the API requests
to make it easier to use. These are located in nena_api.py

In this tutorial the plotting is done using the Bokeh plotting library. Make sure you have
installed this along with the JupyterLab Bokeh extension.

To access data from the API we need to provide our Nena portal username and password. The
root URL is also needed.

We are interested in the %-filling of the Norwegian reservoirs, so we need to know the total
reservoir capacity. This value can be found on NVE's (Norwegian Water Resources and Energy Directorate) [website](https://www.nve.no/energi/analyser-og-statistikk/magasinstatistikk/).

In [ ]:
USER = ""  # Insert your Nena username
PASS = ""  # Insert your Nena password
ROOT = "https://api.nena.no"

In [ ]:
NO_RESERVOIR_CAPACITY = 87200  # The total reservoir capacity in GWh
NO_RES_SERIES = {"NO": "hpddanoresa"}  # Series ID for Norwegian reservoir capacity

In [ ]:
def prep_df(df, relative=True):
    """
    Helper function for preparing dataframe.
    """
    df.index = pd.to_datetime(df.index).isocalendar().week
    df.index.name = "week"
    df = df.reindex(np.arange(1, 53))

    if relative:
        df = df * 100 / NO_RESERVOIR_CAPACITY
    df.reset_index(inplace=True)

    return df


def retrieve_data(series_id, from_date, to_date, keyword):
    """
    Helper function for getting hydrological data.
    """
    data = get_series_data(
        series_id=series_id,
        from_date=from_date,
        to_date=to_date,
        resolution=keyword,
        user_auth=obtain_access_token(username=USER, password=PASS, root_url=ROOT),
    )
    return data


def calc_statistics(df, column, relative=False, reindex=False):
    """
    Calculates weekly median, min, and max statistics 
    on time series.
    
    Args.
    df [DataFrame] : Time series data
    column [str] : Column name to apply statistics to.
    relative [bool] : Data is converted to percentages.
    
    Returns.
    df_stats [DataFrame] : Weekly stats.
    """

    df.index.name = "time"
    df.index = pd.to_datetime(df.index)
    df.reset_index(inplace=True)

    median = df.groupby(df["time"].dt.isocalendar().week)[column].mean()
    minimum = df.groupby(df["time"].dt.isocalendar().week)[column].min()
    maximum = df.groupby(df["time"].dt.isocalendar().week)[column].max()

    df_stats = pd.concat([maximum, median, minimum], axis=1).iloc[:52, :]
    df_stats = pd.concat([df_stats.iloc[35:], df_stats.iloc[:35]])
    df_stats.columns = ["max", "median", "min"]

    if relative:
        df_stats = df_stats * 100 / NO_RESERVOIR_CAPACITY

    df_stats.index.name = "week"
    df_stats.reset_index(inplace=True)

    return df_stats

Here we load the hydrological data. We can use the keywords: "bohy-20" to get the date at the start of the hydrological year (1st of september) 20 years ago (effectivly our data goes back to 2014 only), followed by "eohy-1" which is the end of lasts years hydrological year (2021-08-31).

In [ ]:
df_hydro_history = pd.DataFrame.from_dict(
    retrieve_data(NO_RES_SERIES['NO'], "bohy-20", "eohy-1", "week")["Values"],
    orient="index",
    columns=["capacity"],
)

df_current_year = pd.DataFrame.from_dict(
    retrieve_data(NO_RES_SERIES['NO'], "eohy-1", "last", "week")["Values"],
    orient="index",
    columns=["capacity"],
)

Now we calculate some statistics on the historical hydro data and do some general data prep.
Lastly we merge the data into a single dataframe.

In [ ]:
df_history_stats = calc_statistics(df_hydro_history, "capacity", relative=True)
df_current_year_cleaned = prep_df(df_current_year)
df_tot = pd.merge(
    df_history_stats, df_current_year_cleaned, right_on="week", left_on="week"
)  # Merging historical df with current years data
df_tot = df_tot.sort_values(by="week")

## Plot

Below is an interactive Bokeh plot with the reservoir filling.

In [ ]:
output_notebook()
source = ColumnDataSource(df_tot)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p = figure(tools=TOOLS, width=800, height=400, x_range=(1, 52), y_range=(0, 100))

p.line(
    x="week",
    y="capacity",
    line_color="black",
    line_width=1.5,
    legend_label="2021-22",
    source=source,
)

p.line(
    x="week",
    y="median",
    line_color="red",
    line_dash="dashed",
    legend_label="Median",
    source=source,
    name="line_with_hovertool",
)


p.varea(
    x="week",
    y1="min",
    y2="max",
    source=source,
    fill_alpha=0.1,
    fill_color="red",
    legend_label="Min-Max.",
)


p.title.text = "Norwegian hydro reservoir filling (2014-21)"
p.xgrid[0].grid_line_color = None
p.ygrid[0].grid_line_alpha = 0.5
p.xaxis.axis_label = "Week"
p.yaxis.axis_label = "%"

p.legend.location = "bottom_right"

tooltips = [
    ("Week", "@week"),
    ("Median", "@median"),
    ("Cur. HY", "@capacity"),
    ("Min", "@min"),
    ("Max", "@max"),
]

p.add_tools(HoverTool(names=["line_with_hovertool"], tooltips=tooltips, mode="vline"))
show(p)